## Segmenting and Clustering Neighborhoods in Toronto 

### The data of the toronto -- boroughs and neighborhoods with latitudes and longitudes from the previous question had been stored as 'toronto_coordinates.csv', so as to make the data available for this notebook.


In [56]:
import pandas as pd


In [57]:
# Reading the csv file
toronto = pd.read_csv('toronto_coordinates.csv')
toronto.head()

,Unnamed: 0,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,2,M1E,Scarborough,"Scarborough, Morningside, West Hill",43.763573,-79.188711
3,3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,4,M1H,Scarborough,Scarborough,43.773136,-79.239476


In [58]:
toronto = toronto[['PostalCode', 'Borough', 'Neighbourhood', 'Latitude', 'Longitude']]
toronto.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Scarborough,43.773136,-79.239476


In [23]:
# new_ls =[]
# for ind in range(toronto.shape[0]):
#     value = toronto.loc[ind, 'Borough']
#     if (value.endswith('Toronto')):
#         if value not in new_ls:
#             new_ls.append(value)

# new_df = toronto[toronto['Borough'].isin(new_ls)].reset_index(drop = True)
        


### The segmenting and clustering the neighborhoods of 'Scarborough'

In [59]:
toronto_df = toronto[toronto['Borough'] == 'Scarborough'].reset_index(drop = True)
toronto_df.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Scarborough, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Scarborough,43.773136,-79.239476


In [28]:
toronto_df.shape

(17, 5)

In [62]:
### Importing all the libraries required

import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors

import requests

!conda install -c conda-forge folium=0.5.0 --yes
import folium

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [63]:
## Getting the lati
address = 'Scarborough, Toronto, CA'
geolocator = Nominatim(user_agent = 'ca_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

In [34]:
print(latitude, longitude)

43.773077 -79.257774


In [64]:
map_scarborough = folium.Map(location =[latitude, longitude], zoom_start=12)

for lat,lng,label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker([lat,lng], radius =5, 
                       popup = label, color = 'blue',
                       fill = True, fill_color ='#3186cc',
                       fill_opacity = 0.7, parse_html =False).add_to(map_scarborough)
map_scarborough

In [65]:
# # Define the credentials of Foursquare
CLIENT_ID = 'BISUZFYAQSY1N2JWW44Z35GRQN3FVK22YQPXHVP41KFWZS4T' 
CLIENT_SECRET = 'M4FJBHCIUY0UXRVT3PDBFKMG52FKJEU2OY2OHBZXGY1I3FHV' 
VERSION = '20180605' 

In [66]:
## Exploring the 'Woburn' neighbourhood of 'Scarborough'
toronto_df.loc[3, 'Neighbourhood']
###############   df['Neighbourhood'] = df['Neighbourhood'].replace('\n','', regex=True)

'Woburn'

In [76]:
# Defining the url for getting the info of neighbourhoods 
nbhd_lati = toronto_df.loc[3, 'Latitude']
nbhd_longi = toronto_df.loc[3, 'Longitude']
nbhd_name = toronto_df.loc[3, 'Neighbourhood']

radius = 500
LIMIT = 100

url = "https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}".format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    nbhd_lati,
    nbhd_longi, 
    VERSION, 
    radius, 
    LIMIT)



In [77]:
results = requests.get(url).json()['response']['groups'][0]['items']


In [78]:
from pandas.io.json import json_normalize 

In [79]:
# Function to extract the categories from the json file
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [89]:
#results

In [81]:
# Flattening the json file, cleaning and storing in a dataframe 'nearby_venues'
nearby_venues = json_normalize(results)
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.longitude.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Starbucks,Coffee Shop,43.770037,NaN
1,Tim Hortons,Coffee Shop,43.770827,NaN
2,Korean Grill House,Korean Restaurant,43.770812,NaN
3,Al-Hamd Biryani & Pizza,Indian Restaurant,43.767585,NaN


In [82]:
nearby_venues.shape


(4, 4)

In [83]:
# Repeating the exploration for all the neighbourhoods of 'Scarborough'
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
        
        

In [84]:
scarborough_venues = getNearbyVenues(names = toronto_df['Neighbourhood'],
                                    latitudes = toronto_df['Latitude'],
                                    longitudes = toronto_df['Longitude'])


Rouge, Malvern
Highland Creek, Rouge Hill, Port Union
Scarborough, Morningside, West Hill
Woburn
Scarborough
Scarborough Village
Scarborough, Ionview, Kennedy Park
Clairlea, Golden Mile, Oakridge
Cliffcrest, Cliffside, Scarborough
Birch Cliff, Scarborough
Dorset Park, Scarborough Town Centre, Wexford Heights
Maryvale, Wexford
Agincourt
Scarborough, Scarborough, Tam O'Shanter
Agincourt North, Scarborough, Milliken, Scarborough
Scarborough
Upper Rouge


In [85]:
scarborough_venues.shape

(85, 7)

In [86]:
scarborough_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Rouge, Malvern",43.806686,-79.194353,Wendy's,43.807448,-79.199056,Fast Food Restaurant
1,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
2,"Scarborough, Morningside, West Hill",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
3,"Scarborough, Morningside, West Hill",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
4,"Scarborough, Morningside, West Hill",43.763573,-79.188711,Marina Spa,43.766000,-79.191000,Spa


In [87]:
scarborough_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Agincourt North, Scarborough, Milliken, Scarborough",3,3,3,3,3,3
"Birch Cliff, Scarborough",4,4,4,4,4,4
"Clairlea, Golden Mile, Oakridge",10,10,10,10,10,10
"Cliffcrest, Cliffside, Scarborough",2,2,2,2,2,2
"Dorset Park, Scarborough Town Centre, Wexford Heights",7,7,7,7,7,7
"Highland Creek, Rouge Hill, Port Union",1,1,1,1,1,1
"Maryvale, Wexford",4,4,4,4,4,4
"Rouge, Malvern",1,1,1,1,1,1


In [88]:
# Getting the total different categories in the neighbourhoods of 'Scarborough'
len(scarborough_venues['Venue Category'].unique())

51

### Now, Analyze each neighbourhood of 'Scarborough'

In [93]:
# One hot encoding converts the categorical column 'Venue Category' to the various numerical col representing each category
## There will be 51 columns created as there are 51 categories

scar_onehot = pd.get_dummies(scarborough_venues[['Venue Category']], prefix="", prefix_sep="")

scar_onehot['Neighborhood'] = scarborough_venues['Neighborhood']
new_columns = [scar_onehot.columns[-1]] + list(scar_onehot.columns[:-1])

scar_onehot = scar_onehot[new_columns]
scar_onehot.head()

,Neighborhood,American Restaurant,Athletics & Sports,Bakery,Bank,Bar,Breakfast Spot,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Convenience Store,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,General Entertainment,Grocery Store,Hakka Restaurant,Hobby Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Skating Rink,Smoke Shop,Soccer Field,Spa,Thai Restaurant,Thrift / Vintage Store,Vietnamese Restaurant
0,"Rouge, Malvern",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Highland Creek, Rouge Hill, Port Union",0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Scarborough, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
3,"Scarborough, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Scarborough, Morningside, West Hill",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0


In [94]:
scar_group = scar_onehot.groupby('Neighborhood').mean().reset_index()

In [95]:
scar_group.shape

(15, 52)

#### Creating a dataframe to store the top 10 venues.
1. Define a function which returns the venues soreted in descending order (first 10) will be returned
1. First create an empty dataframe with relevant column names
1. Storing the contents to the created dataframe

In [102]:
## Define the function return_top10 

def return_top10(row, top_venues):
    rows = row.iloc[1:]
    rows_sorted = rows.sort_values(ascending=False)
    return rows_sorted.index.values[0:top_venues]


#Creating a new dataframe
top_venues = 10
indicators = ['st', 'nd', 'rd']
columns = ['Neighbourhood']

# naming the column names as 1st , 2nd , 3rd, 4th top venues

for i in np.arange(top_venues):
    try:
        columns.append('{}{} Top Venue'.format(i+1, indicators[i]))
    except:
        columns.append('{}th Top Venue'.format(i+1))

sorted_venues = pd.DataFrame(columns = columns)
sorted_venues['Neighbourhood'] = scar_group['Neighborhood']

for i in np.arange(scar_group.shape[0]):
    sorted_venues.iloc[i,1:] = return_top10(scar_group.iloc[i,:], top_venues)

sorted_venues.head()


,Neighbourhood,1st Top Venue,2nd Top Venue,3rd Top Venue,4th Top Venue,5th Top Venue,6th Top Venue,7th Top Venue,8th Top Venue,9th Top Venue,10th Top Venue
0,Agincourt,Clothing Store,Skating Rink,Breakfast Spot,Lounge,Coffee Shop,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store
1,"Agincourt North, Scarborough, Milliken, Scarbo...",Park,Playground,Clothing Store,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store,Department Store
2,"Birch Cliff, Scarborough",General Entertainment,Skating Rink,Café,College Stadium,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,Fried Chicken Joint,Fast Food Restaurant
3,"Clairlea, Golden Mile, Oakridge",Bus Line,Bakery,Intersection,Fast Food Restaurant,Metro Station,Bus Station,Park,Soccer Field,Breakfast Spot,Bar
4,"Cliffcrest, Cliffside, Scarborough",American Restaurant,Motel,Indian Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store


### Clustering the neighbourhoods of 'Scarborough'
### The neighbourhood is clustered into 3 clusters

In [158]:
k = 4
scar_cluster = scar_group.drop('Neighborhood', 1)

clusters = KMeans(n_clusters=k, random_state=0).fit(scar_cluster)
labels = clusters.labels_

In [159]:
sorted_venues.drop(['Cluster Labels'], 1, inplace=True)

In [160]:
# including the labels as the column 'cluster labels' in the 'sorted_venues' dataframe
sorted_venues.insert(0, 'Cluster Labels', labels)
scarborough_final = toronto_df
scarborough_final = scarborough_final.join(sorted_venues.set_index('Neighbourhood'), on='Neighbourhood')
scarborough_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Top Venue,2nd Top Venue,3rd Top Venue,4th Top Venue,5th Top Venue,6th Top Venue,7th Top Venue,8th Top Venue,9th Top Venue,10th Top Venue
0,M1B,Scarborough,"Rouge, Malvern",43.81,-79.19,3.00,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78,-79.16,2.00,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Scarborough, Morningside, West Hill",43.76,-79.19,1.00,Intersection,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,General Entertainment
3,M1G,Scarborough,Woburn,43.77,-79.22,1.00,Coffee Shop,Indian Restaurant,Korean Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Scarborough,43.77,-79.24,1.00,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Pizza Place,Fried Chicken Joint,Thrift / Vintage Store,Coffee Shop,Caribbean Restaurant,Grocery Store,Pharmacy


In [161]:
labels

array([1, 1, 1, 1, 0, 1, 2, 1, 3, 1, 1, 1, 1, 1, 1], dtype=int32)

In [162]:
pd.options.display.float_format = '{:,.2f}'.format
scarborough_final.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Top Venue,2nd Top Venue,3rd Top Venue,4th Top Venue,5th Top Venue,6th Top Venue,7th Top Venue,8th Top Venue,9th Top Venue,10th Top Venue
0,M1B,Scarborough,"Rouge, Malvern",43.81,-79.19,3.00,Fast Food Restaurant,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Electronics Store,Discount Store,Department Store
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.78,-79.16,2.00,Bar,Vietnamese Restaurant,Coffee Shop,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
2,M1E,Scarborough,"Scarborough, Morningside, West Hill",43.76,-79.19,1.00,Intersection,Spa,Electronics Store,Breakfast Spot,Rental Car Location,Pizza Place,Medical Center,Mexican Restaurant,Coffee Shop,General Entertainment
3,M1G,Scarborough,Woburn,43.77,-79.22,1.00,Coffee Shop,Indian Restaurant,Korean Restaurant,Hakka Restaurant,Grocery Store,General Entertainment,Fried Chicken Joint,Fast Food Restaurant,Electronics Store,Discount Store
4,M1H,Scarborough,Scarborough,43.77,-79.24,1.00,Fast Food Restaurant,Chinese Restaurant,American Restaurant,Pizza Place,Fried Chicken Joint,Thrift / Vintage Store,Coffee Shop,Caribbean Restaurant,Grocery Store,Pharmacy


### Visulaize the clusters of the venues of the different neighbourhood of 'Scarborough'

In [163]:
scarborough_final.dropna(inplace=True)

In [164]:
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 10)

x = np.arange(k)
y = [i + x + (i+x) ** 2 for i in range(k)]

colors_array = cm.rainbow(np.linspace(0,1,len(y)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

#print(rainbow)

for lat, lng, nbhd, cluster in zip(scarborough_final['Latitude'],
                                   scarborough_final['Longitude'],
                                   scarborough_final['Neighbourhood'],
                                   scarborough_final['Cluster Labels']):
    #print(cluster)
    label = folium.Popup(str(nbhd) + '  Cluster' + str(int(cluster)), parse_html=True)
    folium.CircleMarker([lat,lng], radius=5,
                       popup=label, color=rainbow[int(cluster)-1],
                       fill=True, fill_color=rainbow[int(cluster)-1],
                       fill_opacity=0.7).add_to(map_clusters)
    
map_clusters